In [ ]:
from flask import Flask, render_template, request, jsonify, session, send_from_directory
from database import (
    init_db, hash_password, create_user, get_user_by_email, get_user_by_id,
    get_all_posts, create_post, toggle_reaction, create_comment, save_chat_message,
    get_chat_history, get_chat_sessions, delete_chat_session, update_user_profile_picture,
    save_search_history, save_veterinarian_details, get_search_history, get_search_results,
    get_posts_by_cluster, get_all_clusters
)
from flask_cors import CORS
from vet import GoogleMapsScraper
import threading
import uuid
import time
import atexit
from datetime import datetime
from core import chatbot_db, get_chatbot_response
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.config['SECRET_KEY'] = 'cat-community-secret-key-2024'
app.config['UPLOAD_FOLDER'] = 'static/uploads/profile_pictures'
app.config['POST_UPLOAD_FOLDER'] = 'static/uploads/post_images'
app.config['MAX_CONTENT_LENGTH'] = 2 * 1024 * 1024  
app.config['ALLOWED_EXTENSIONS'] = {'png', 'jpg', 'jpeg', 'gif'}

SERVER = 'localhost' 
DATABASE = 'cats_db'
app.config['SQLALCHEMY_DATABASE_URI'] = f'mssql+pyodbc://@{SERVER}/{DATABASE}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

CORS(app)

active_searches = {}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

def save_profile_picture(file, user_id):
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_extension = filename.rsplit('.', 1)[1].lower()
        new_filename = f"user_{user_id}_{int(time.time())}.{file_extension}"
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], new_filename)
        file.save(file_path)
        return new_filename
    return None

def save_post_image(file, user_id):
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_extension = filename.rsplit('.', 1)[1].lower()
        new_filename = f"post_{user_id}_{int(time.time())}.{file_extension}"
        file_path = os.path.join(app.config['POST_UPLOAD_FOLDER'], new_filename)
        file.save(file_path)
        return new_filename
    return None

def background_search(search_uuid, location, user_id=None):
    scraper = None
    try:
        active_searches[search_uuid]['status'] = 'searching'
        active_searches[search_uuid]['progress'] = 40
        
        scraper = GoogleMapsScraper()
        
        active_searches[search_uuid]['status'] = 'scraping'
        active_searches[search_uuid]['progress'] = 70
        clinics = scraper.search_veterinarians(location)
        
        active_searches[search_uuid]['results'] = clinics
        active_searches[search_uuid]['status'] = 'completed'
        active_searches[search_uuid]['progress'] = 100
        active_searches[search_uuid]['clinics_found'] = len(clinics)
        
        if user_id:
            try:
                save_search_history(user_id, search_uuid, location, len(clinics))
                for clinic in clinics:
                    save_veterinarian_details(search_uuid, clinic)
            except Exception:
                pass
    except Exception as e:
        active_searches[search_uuid]['status'] = 'failed'
        active_searches[search_uuid]['error'] = str(e)
    finally:
        if scraper:
            try:
                scraper.close()
            except Exception:
                pass

@app.route('/api/signup', methods=['POST'])
def signup():
    try:
        if request.content_type.startswith('multipart/form-data'):
            first_name = request.form.get('first_name', '').strip()
            last_name = request.form.get('last_name', '').strip()
            email = request.form.get('email', '').strip().lower()
            password = request.form.get('password', '')
            gender = request.form.get('gender', 'male')
            profile_picture = request.files.get('profile_picture')
        else:
            data = request.get_json()
            first_name = data.get('first_name', '').strip()
            last_name = data.get('last_name', '').strip()
            email = data.get('email', '').strip().lower()
            password = data.get('password', '')
            gender = data.get('gender', 'male')
            profile_picture = None
        
        if not all([first_name, last_name, email, password]):
            return jsonify({'error': 'All fields are required'}), 400
        
        if len(first_name) < 2 or len(last_name) < 2:
            return jsonify({'error': 'First and last name must be at least 2 characters long'}), 400
        
        if len(password) < 6:
            return jsonify({'error': 'Password must be at least 6 characters long'}), 400
        
        if '@' not in email or '.' not in email:
            return jsonify({'error': 'Please enter a valid email address'}), 400
        
        if gender not in ['male', 'female']:
            gender = 'male'
        
        user_id = create_user(first_name, last_name, email, password, gender)
        
        profile_pic_filename = None
        if profile_picture:
            saved_filename = save_profile_picture(profile_picture, user_id)
            if saved_filename:
                profile_pic_filename = saved_filename
                update_user_profile_picture(user_id, profile_pic_filename)
        
        user_data = get_user_by_id(user_id)
        
        return jsonify({
            'message': 'User created successfully',
            'user': {
                'id': user_data['id'],
                'first_name': user_data['first_name'],
                'last_name': user_data['last_name'],
                'email': user_data['email'],
                'gender': user_data['gender'],
                'profile_picture': user_data['profile_picture']
            }
        }), 201
        
    except ValueError as e:
        return jsonify({'error': str(e)}), 400
    except Exception:
        return jsonify({'error': 'Something went wrong. Please try again.'}), 500

@app.route('/api/login', methods=['POST'])
def login():
    try:
        data = request.get_json()
        email = data.get('email', '').strip().lower()
        password = data.get('password', '')
        
        if not email or not password:
            return jsonify({'error': 'Email and password are required'}), 400
        
        user = get_user_by_email(email)
        
        if user and user['password'] == hash_password(password):
            session['user_id'] = user['id']
            session['user_name'] = f"{user['first_name']} {user['last_name']}"
            session['profile_picture'] = user['profile_picture']
            
            return jsonify({
                'message': 'Login successful',
                'user': {
                    'id': user['id'],
                    'first_name': user['first_name'],
                    'last_name': user['last_name'],
                    'email': email,
                    'gender': user['gender'],
                    'profile_picture': user['profile_picture']
                }
            }), 200
        else:
            return jsonify({'error': 'Invalid email or password'}), 401
            
    except Exception:
        return jsonify({'error': 'Something went wrong. Please try again.'}), 500

@app.route('/api/logout', methods=['POST'])
def logout():
    session.clear()
    return jsonify({'message': 'Logged out successfully'}), 200

@app.route('/api/update_profile_picture', methods=['POST'])
def update_profile_picture():
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to update profile picture'}), 401
        
        if 'profile_picture' not in request.files:
            return jsonify({'error': 'No file provided'}), 400
        
        file = request.files['profile_picture']
        if file.filename == '':
            return jsonify({'error': 'No file selected'}), 400
        
        user_id = session['user_id']
        saved_filename = save_profile_picture(file, user_id)
        
        if saved_filename:
            update_user_profile_picture(user_id, saved_filename)
            session['profile_picture'] = saved_filename
            
            return jsonify({
                'message': 'Profile picture updated successfully',
                'profile_picture': saved_filename
            }), 200
        else:
            return jsonify({'error': 'Invalid file type. Please upload PNG, JPG, or GIF.'}), 400
        
    except Exception:
        return jsonify({'error': 'Failed to update profile picture'}), 500

@app.route('/static/uploads/profile_pictures/<filename>')
def serve_profile_picture(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@app.route('/api/posts', methods=['GET'])
def get_posts():
    try:
        cluster = request.args.get('cluster', 'all')
        
        if cluster == 'all':
            posts_data = get_all_posts()
        else:
            posts_data = get_posts_by_cluster(cluster)
            
        return jsonify(posts_data), 200
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/clusters', methods=['GET'])
def get_clusters():
    try:
        clusters = get_all_clusters()
        return jsonify({'clusters': clusters}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/static/uploads/post_images/<filename>')
def serve_post_image(filename):
    return send_from_directory(app.config['POST_UPLOAD_FOLDER'], filename)

@app.route('/api/create_post', methods=['POST'])
def create_post_route():
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to post'}), 401
            
        if request.content_type and request.content_type.startswith('multipart/form-data'):
            content = request.form.get('content', '').strip()
            post_type = request.form.get('post_type', 'other')
            cluster = request.form.get('cluster', 'general')
            post_image = request.files.get('post_image')
        else:
            data = request.get_json()
            content = data.get('content', '').strip()
            post_type = data.get('post_type', 'other')
            cluster = data.get('cluster', 'general')
            post_image = None
        
        if not content:
            return jsonify({'error': 'Post content is required'}), 400
        
        if len(content) > 1000:
            return jsonify({'error': 'Post content is too long (max 1000 characters)'}), 400
        
        valid_post_types = ['inquiry', 'lost_cat', 'advice', 'story', 'help', 'other']
        if post_type not in valid_post_types:
            post_type = 'other'
        
        post_image_filename = None
        if post_image and allowed_file(post_image.filename):
            saved_filename = save_post_image(post_image, session['user_id'])
            if saved_filename:
                post_image_filename = saved_filename
        
        post_id = create_post(content, session['user_name'], session['user_id'], post_type, post_image_filename, cluster)
        
        return jsonify({
            'message': 'Post created successfully',
            'post': {
                'id': post_id,
                'content': content,
                'author': session['user_name'],
                'author_picture': session.get('profile_picture', 'default.png'),
                'post_type': post_type,
                'post_image': post_image_filename,
                'cluster': cluster
            }
        }), 201
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/react/<int:post_id>', methods=['POST'])
def react_to_post(post_id):
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to react to posts'}), 401
        
        data = request.get_json()
        reaction_type = data.get('reaction_type', 'like')
        
        valid_reactions = ['like', 'love', 'haha', 'wow', 'sad', 'angry']
        if reaction_type not in valid_reactions:
            return jsonify({'error': 'Invalid reaction type'}), 400
        
        result = toggle_reaction(session['user_id'], post_id, reaction_type)
        
        return jsonify(result), 200
            
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/add_comment', methods=['POST'])
def add_comment():
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to comment'}), 401
            
        data = request.get_json()
        post_id = data.get('post_id')
        text = data.get('text', '').strip()
        
        if not post_id or not text:
            return jsonify({'error': 'Post ID and comment text are required'}), 400
        
        if len(text) > 500:
            return jsonify({'error': 'Comment is too long (max 500 characters)'}), 400
        
        comment_count = create_comment(text, session['user_name'], session['user_id'], post_id)
        
        return jsonify({
            'message': 'Comment added successfully',
            'comment': {
                'text': text,
                'author': session['user_name'],
                'author_picture': session.get('profile_picture', 'default.png')
            },
            'count': comment_count
        }), 201
        
    except ValueError as e:
        return jsonify({'error': str(e)}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/search', methods=['POST'])
def search_clinics():
    try:
        data = request.get_json()
        location = data.get('location', '').strip()
        
        if not location:
            return jsonify({'error': 'Please enter a location'}), 400
        
        search_uuid = str(uuid.uuid4())
        
        user_id = session.get('user_id')
        
        active_searches[search_uuid] = {
            'location': location,
            'status': 'started',
            'progress': 0,
            'started_at': datetime.now(),
            'results': None,
            'clinics_found': 0,
            'user_id': user_id
        }
        
        search_thread = threading.Thread(
            target=background_search,
            args=(search_uuid, location, user_id)
        )
        search_thread.daemon = True
        search_thread.start()
        
        return jsonify({
            'search_uuid': search_uuid,
            'status': 'started',
            'message': f'Search for "{location}" started in background',
            'location': location
        })
        
    except Exception as e:
        return jsonify({'error': f'Failed to start search: {str(e)}'}), 500

@app.route('/search/status/<search_uuid>', methods=['GET'])
def get_search_status_api(search_uuid):
    try:
        if search_uuid not in active_searches:
            return jsonify({'error': 'Search not found'}), 404
        
        search_info = active_searches[search_uuid]
        
        if search_info['status'] == 'started':
            elapsed = time.time() - search_info['started_at'].timestamp()
            if elapsed > 30:
                search_info['status'] = 'searching'
                search_info['progress'] = 40
            elif elapsed > 15:
                search_info['status'] = 'searching'
                search_info['progress'] = 20
            else:
                search_info['progress'] = 10
        
        return jsonify({
            'search_uuid': search_uuid,
            'location': search_info['location'],
            'status': search_info['status'],
            'progress': search_info['progress'],
            'clinics_found': search_info.get('clinics_found', 0)
        })
        
    except Exception as e:
        return jsonify({'error': f'Failed to get status: {str(e)}'}), 500

@app.route('/search/results/<search_uuid>', methods=['GET'])
def get_search_results_api(search_uuid):
    try:
        if search_uuid not in active_searches:
            return jsonify({'error': 'Search not found'}), 404
        
        search_info = active_searches[search_uuid]
        
        if search_info['status'] != 'completed':
            return jsonify({'error': 'Search not completed yet'}), 400
        
        clinics = search_info.get('results', [])
        
        return jsonify({
            'search_uuid': search_uuid,
            'location': search_info['location'],
            'clinics_found': len(clinics),
            'clinics': clinics,
            'status': 'completed'
        })
        
    except Exception as e:
        return jsonify({'error': f'Failed to get results: {str(e)}'}), 500

@app.route('/api/history', methods=['GET'])
def get_history():
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to view history'}), 401
        
        history = get_search_history(session['user_id'])
        return jsonify({'history': history}), 200
        
    except Exception as e:
        return jsonify({'error': f'Failed to get history: {str(e)}'}), 500

@app.route('/api/history/<search_uuid>', methods=['GET'])
def get_saved_search_results(search_uuid):
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to view saved searches'}), 401
        
        results = get_search_results(search_uuid)
        if not results:
            return jsonify({'error': 'Saved search not found'}), 404
        
        return jsonify(results), 200
        
    except Exception as e:
        return jsonify({'error': f'Failed to get saved search: {str(e)}'}), 500

@app.route('/chatbot/message', methods=['POST'])
def chatbot_message():
    try:
        data = request.get_json()
        message = data.get('message', '').strip()
        session_id = data.get('session_id', 'default')
        
        if not message:
            return jsonify({'error': 'Message is required'}), 400
        
        response = get_chatbot_response(message, chatbot_db)
        
        if 'user_id' in session:
            try:
                save_chat_message(session['user_id'], message, response, True, session_id)
                save_chat_message(session['user_id'], message, response, False, session_id)
            except Exception:
                pass
        
        return jsonify({
            'response': response,
            'session_id': session_id,
            'timestamp': time.time()
        })
        
    except Exception as e:
        return jsonify({'error': f'Chatbot error: {str(e)}'}), 500

@app.route('/chatbot/summary', methods=['GET'])
def chatbot_summary():
    try:
        summary_response = get_chatbot_response("write a concise summary of the file in simple clear language", chatbot_db)
        key_points_response = get_chatbot_response("list the key points from the file as bullet points", chatbot_db)
        
        return jsonify({
            'summary': summary_response,
            'key_points': key_points_response
        })
        
    except Exception as e:
        return jsonify({'error': f'Failed to get summary: {str(e)}'}), 500

@app.route('/api/chat/history', methods=['GET'])
def get_chat_history_route():
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to view chat history'}), 401
        
        session_id = request.args.get('session_id')
        user_id = session['user_id']
        
        if session_id:
            messages = get_chat_history(user_id, session_id)
        else:
            messages = get_chat_history(user_id)
        
        return jsonify({'messages': messages}), 200
        
    except Exception as e:
        return jsonify({'error': f'Failed to get chat history: {str(e)}'}), 500

@app.route('/api/chat/sessions', methods=['GET'])
def get_chat_sessions_route():
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to view chat sessions'}), 401
        
        user_id = session['user_id']
        sessions = get_chat_sessions(user_id)
        
        return jsonify({'sessions': sessions}), 200
        
    except Exception as e:
        return jsonify({'error': f'Failed to get chat sessions: {str(e)}'}), 500

@app.route('/api/chat/session/<session_id>', methods=['DELETE'])
def delete_chat_session_route(session_id):
    try:
        if 'user_id' not in session:
            return jsonify({'error': 'Please login to delete chat sessions'}), 401
        
        user_id = session['user_id']
        deleted_count = delete_chat_session(user_id, session_id)
        
        return jsonify({
            'message': f'Deleted {deleted_count} messages from session',
            'deleted_count': deleted_count
        }), 200
        
    except Exception as e:
        return jsonify({'error': f'Failed to delete chat session: {str(e)}'}), 500

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/health')
def health_check():
    chatbot_status = "available" if chatbot_db else "unavailable"
    return jsonify({
        'status': 'healthy', 
        'message': 'Meow Cat Care Platform is running',
        'active_searches': len(active_searches),
        'chatbot_status': chatbot_status,
        'database': 'SQL Server with Windows Authentication'
    })

@app.route('/active-searches')
def get_active_searches():
    active = []
    for search_uuid, info in active_searches.items():
        active.append({
            'search_uuid': search_uuid,
            'location': info['location'],
            'status': info['status'],
            'progress': info.get('progress', 0),
            'user_id': info.get('user_id')
        })
    
    return jsonify({'active_searches': active})

def cleanup():
    for search_uuid, info in active_searches.items():
        if info['status'] in ['started', 'searching', 'scraping']:
            info['status'] = 'cancelled'

atexit.register(cleanup)

if __name__ == '__main__':
    try:
        init_db()
    except Exception:
        pass
    
    app.run(debug=True, port=5001, use_reloader=False)

d:\Depi\Final Project\Cat Lovers\project10 SQL Server\core.py:125: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Mohammed\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [25/Nov/2025 00:54:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:23] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:23] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Nov/2025 00:54:42] "POST /api/signup HTTP/1.1" 201 -
127.0.0.1 - - [25/Nov/2025 00:54:49] "POST /api/login HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:50] "GET /api/clusters HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:50] "GET /static/uploads/profile_pictures/male.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:50] "GET /api/posts HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:50] "GET /static/uploads/post_images/post_2_1764023741.jpg HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:52] "GET /api/clusters HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 00:54:52] "GET /api/posts HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/